In [1]:
import rospy
import numpy as np
from panda_robot_custom import PandaRobotCustom
from tf.transformations import quaternion_from_euler, euler_from_quaternion
import my_log
import tools

log = my_log.logger()
log._print_logger_info()

rospy.init_node("franka_test", anonymous=True)

robot = PandaRobotCustom()
rospy.sleep(1) 

2023-08-21 16:44:46 [INFO] Logger 'my_log' with level 20
2023-08-21 16:44:46 [INFO] Initializing PandaRobot class
2023-08-21 16:44:46 [INFO] Setting stiffness
2023-08-21 16:44:46 [INFO] Old stiffness: {'translational_stiffness': 200.0, 'rotational_stiffness': 10.0, 'nullspace_stiffness': 0.5}
2023-08-21 16:44:48 [INFO] New stiffness: {'translational_stiffness': 400.0, 'rotational_stiffness': 20.0, 'nullspace_stiffness': 1.0}


/opt/ros/noetic/lib/python3/dist-packages/tf/listener.py:97: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(self._buffer.all_frames_as_yaml()) or {}


2023-08-21 16:44:50 [INFO] Trans:[0.3063133259334194, 5.904197867475839e-05, 0.5969419556999807], Rot: (3.0737737271366083, 0.05511593807055255, -0.7870335014390171)
2023-08-21 16:44:50 [INFO] q_states: (-0.0021356985330287515, -0.7474784951107871, 0.0036164474550925974, -2.297837409193419, -0.0064508444565989365, 1.4635141803913223, 0.7851583162608646)
2023-08-21 16:44:53 [INFO] Panda initialization DONE


In [9]:
trans = [0.6, 0.0, 0.5]
rot = [np.pi, -np.pi/9, 0.0]
robot.cart_move(trans, quaternion_from_euler(rot[0], rot[1], rot[2]))

In [8]:
log.info(robot.get_cart_pose())

2023-08-21 16:54:09 [INFO] ([0.5994092859799204, 0.000124147672532853, 0.5258246300916453], [-0.9998548040788275, -0.0010412004005530273, -0.010213693521316396, -0.013600262014667526])


/opt/ros/noetic/lib/python3/dist-packages/tf/listener.py:97: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(self._buffer.all_frames_as_yaml()) or {}
